In [1]:
# !pip install --upgrade pip
# !pip install llama-index
# !pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface
# !pip install llama_index==0.9.24

In [2]:
# from llama_index.llms.ollama import Ollama

# llm = Ollama(model="llama2", request_timeout=600) #base_url = 'http://localhost:11434',

# response = llm.complete("Do you know about Claude-3?") ## stream_complete() for streaming response

# response
# print(response.text)

In [3]:
# Importing required modules
from pypdf import PdfReader
from llama_index.core import VectorStoreIndex, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
# from llama_index import SimpleDirectoryReader

# Creating a pdf reader object
reader = PdfReader('faq.pdf')
# reader = SimpleDirectoryReader("/home/abhi/Documents/ChatBot/Llama2/data").load_data()

# Extracting text from all pages
all_text = ""
for page in reader.pages:
    all_text += page.extract_text() + "\n"

# Setting up the embedding model and LLM
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
Settings.llm = Ollama(model="llama2", request_timeout=600)

# Creating a Document object
doc = Document(text=all_text)

# Creating the VectorStoreIndex
index = VectorStoreIndex.from_documents([doc])

/home/abhi/Documents/ChatBot/Llama2/my_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
retriever = index.as_retriever(similarity_top_k=2)

# Function to get response with LLM validation
def get_validated_response(question):
    # Retrieve relevant documents
    retrieved_nodes = retriever.retrieve(question)
    
    # Prepare context for LLM
    context = "\n".join([node.text for node in retrieved_nodes])
    
    # Prepare prompt for LLM
    prompt = f"""Given the following context and question, provide a accurate and concise answer. If the context doesn't contain enough information to answer the question confidently, state that clearly.

Context:
{context}

Question: {question}

Answer:"""

    # Use LLM to generate a validated response
    llm = Ollama(model="llama2", request_timeout=600, temperature=0.1)
    response = llm.complete(prompt)
    
    return response

# Example usage
# question = "i am a farmar, how can i apply for this scheme??"
# response = get_validated_response(question)
# print(response)

In [ ]:
question = "benefits of PMAY-U 2.0"
response = get_validated_response(question)
print(response)

The implementation period of PMAY-U 2.0 is from 2024 to 2029, and it aims to provide central assistance to 1 crore eligible families/beneficiaries through states/UTs/PLIs to construct, purchase or rent affordable houses in urban areas over a period of 5 years. The scheme will be implemented through Unified Web Portal/Management Information System (MIS) developed in the Scheme, which will have a real-time progress on key performance indicators, dashboards, and analytics to enable informed decision-making by program managers and policymakers.

The eligibility criteria for seeking benefit under PMAY-U 2.0 are families belonging to EWS/LIG/MIG segments, living in urban areas, having no pucca house either in his/her name or in the name of any member of his/her family anywhere in the country. The houses sanctioned under PMAY-U which got curtailed by CSMC after 31.12.2023 due to any reason on the recommendation of States/UTs, houses will not be sanctioned in PMAY-U 2.0 to those beneficiaries.